# Project: 멋진 챗봇 만들기

지난 노드에서 챗봇과 번역기는 같은 집안이라고 했던 말을 기억하시나요?  
앞서 배운 Seq2seq번역기와 Transfomer번역기에 적용할 수도 있겠지만, 이번 노드에서 배운 번역기 성능 측정법을 챗봇에도 적용해 봅시다.

Step 1. 데이터 다운로드
아래 링크에서 ChatbotData.csv 를 다운로드해 챗봇 훈련 데이터를 확보합니다. csv 파일을 읽는 데에는 pandas 라이브러리가 적합합니다. 읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장하세요!

## Step 1. 데이터 다운로드

In [1]:
! sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
! python3 -m pip install --upgrade pip
! python3 -m pip install konlpy
! sudo apt-get install curl git
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.16).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
python3-dev is already the newest version (3.6.7-1~18.04).
openjdk-8-jdk is already the newest version (8u292-b10-0ubuntu1~18.04).
python3-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.5).
0 upgraded, 0 newly installed, 0 to remove and 86 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.16).
git is already the newest version (1:2.17.1-1ubuntu0.9).
0 upgraded, 0 newly installed, 0 to remove and 86 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [2]:
import re
import pandas as pd
import os
import numpy as np
import random
import gensim
from konlpy.tag import Mecab
from tqdm import tqdm_notebook


In [3]:
file_path = os.getenv("HOME") + '/aiffel/transformer_chatbot/data/ChatbotData .csv'

dataset = pd.read_csv(file_path)
dataset.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [4]:
questions, answers = dataset['Q'], dataset['A']

In [5]:
questions

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

## Step 2. 데이터 정제
1. 영문자의 경우, 모두 소문자로 변환합니다.  
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.  
문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮습니다.

In [6]:
def preprocess_sentence(dataset):
    dataset = dataset.lower() # 영문자 소문자로 변환
    re.sub(r"[^a-zA-Zㅏ-ㅣㄱ-ㅎ-가-힣0-9,.?!]+", " ", dataset)
    return dataset

## Step 3. 데이터 토큰화

토큰화에는 KoNLPy의 mecab 클래스를 사용합니다.  
1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.  
2. 데이터를 앞서 정의한 `preprocess_sentence()` 함수로 정제하고, 토큰화합니다.  
3. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 `mecab.morphs` 함수를 전달하면 됩니다.  
4. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외됩니다.  
5. 중복되는 문장은 데이터에서 제외합니다. 소스:타겟 쌍을 비교하지 않고 소스는 소스대로, 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!  
      
구현한 함수를 활용하여 `questions`와 `answers`를 각각 `que_corpus,` `ans_corpus`에 토큰화하여 저장합니다.

In [63]:
def build_corpus(src_sentence, tgt_sentence):
    tokenized_src, tokenized_len_src = [], []
    tokenized_tgt, tokenized_len_tgt = [], []
    for src, tgt in zip(src_sentence, tgt_sentence):
        tokenizer = Mecab()
        mecab_src = tokenizer.morphs(preprocess_sentence(src))
        mecab_tgt = tokenizer.morphs(preprocess_sentence(tgt))
        
        tokenized_src.append(mecab_src)
        tokenized_tgt.append(mecab_tgt)
        
        tokenized_len_src.append(len(mecab_src))
        tokenized_len_tgt.append(len(mecab_tgt))
        
    total_length = tokenized_len_src + tokenized_len_tgt
    
    max_len = np.max(total_length)
    mean_len = np.mean(total_length)
    mid_len = np.median([mean_len, max_len])
    print(f'mid_len : {mid_len}')
        
    src_corpus, tgt_corpus = [], []
    for s, t in zip(tokenized_src, tokenized_tgt):
        if len(s) < mid_len and s not in tokenized_src:
            tokenized_src.append(s)
            
        if len(t) < mid_len and t not in tokenized_tgt:
            tokenized_tgt.append(t)
            
    return tokenized_src, tokenized_tgt

In [64]:
que_corpus, ans_corpus = build_corpus(questions, answers)

mid_len : 23.852046857819506


In [65]:
print(que_corpus[:5], "\n",ans_corpus[:5])

[['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'], ['ppl', '심하', '네']] 
 [['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['눈살', '이', '찌푸려', '지', '죠', '.']]


In [66]:
len(que_corpus), len(ans_corpus)

(11823, 11823)

## Step 4. Augmentation
우리에게 주어진 데이터는 1만 개가량으로 적은 편에 속합니다. 이럴 때에 사용할 수 있는 테크닉을 배웠으니 활용해 봐야겠죠? Lexical Substitution을 실제로 적용해 보도록 하겠습니다.  
  
링크를 참고하여 한국어로 사전 훈련된 Embedding 모델을 다운로드합니다. `Korean(w)` 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 `Korean(w)`를 찾아 다운로드하고, `ko.bin` 파일을 얻으세요!  
  
다운로드한 모델을 활용해 데이터를 Augmentation 하세요! 앞서 정의한 `lexical_sub()` 함수를 참고하면 도움이 많이 될 겁니다.  
  
Augmentation된 `que_corpus` 와 원본 `ans_corpus` 가 병렬을 이루도록, 이후엔 반대로 원본 `que_corpus` 와 Augmentation된 `ans_corpus` 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배 가량으로 늘어나도록 합니다.

In [67]:
# 4.0.X 버전부터는 vocab 코드가 패키지 안에서 사라지기 때문에 ko.bin을 로드하기
# 위해서는 다운그레이드를 해줘야 함
# pip install --upgrade gensim==3.8.3

In [68]:
ko_bin_path = os.getenv('HOME') + '/aiffel/NLPGD/GD6/ko.bin'
word2vec = gensim.models.Word2Vec.load(ko_bin_path)

In [69]:
def lexical_sub(sentence, word2vec):
    try: 
        _from = random.choice(sentence) # sentence에서 하나의 형태소를 무작위로 고름
        _to = word2vec.most_similar(_from)[0][0] # 가장 비슷한 단어를 추출
    except: # sentence 문장이 word2vec에 없으면 그냥 sentence를 반환
        return sentence
    
    res = []
    for x in sentence:
        if x is _from: res.append(_to)
        else: res.append(x)
            
    return res

In [70]:
a = '싫'
b = word2vec.most_similar(a)
b

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('힘들', 0.6567603349685669),
 ('귀찮', 0.634676456451416),
 ('일쑤', 0.6083942651748657),
 ('편하', 0.6059445738792419),
 ('꺼리', 0.5969125628471375),
 ('길들이', 0.5821220874786377),
 ('어렵', 0.5785283446311951),
 ('풀어쓰', 0.570091187953949),
 ('아깝', 0.5619203448295593),
 ('웃기', 0.5485275983810425)]

In [71]:
que_corpus[1]

['1', '지망', '학교', '떨어졌', '어']

In [72]:
arg_que_corpus = [lexical_sub(x, word2vec) for x in que_corpus]
arg_ans_corpus = [lexical_sub(x, word2vec) for x in ans_corpus]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


In [73]:
for i in range(5):
    print("Q:", " ".join(que_corpus[i]), "/" , " ".join(arg_que_corpus[i]))
    print("A:", " ".join(ans_corpus[i]), "/" , " ".join(arg_ans_corpus[i]))

Q: 12 시 땡 ! / 12 시 땡 !
A: 하루 가 또 가 네요 . / 하루 놀드 또 가 네요 .
Q: 1 지망 학교 떨어졌 어 / 1 지망 학교의 떨어졌 어
A: 위로 해 드립니다 . / 위로 해 드립니다 는데
Q: 3 박 4 일 놀 러 가 고 싶 다 / 3 박 4 일 놀 러 가 기에 싶 다
A: 여행 은 언제나 좋 죠 . / 여행 은 언제나 좋 죠 는데
Q: 3 박 4 일 정도 놀 러 가 고 싶 다 / 3 박 4 일 정도 놀 ㄹ래 가 고 싶 다
A: 여행 은 언제나 좋 죠 . / 여행 은데 언제나 좋 죠 .
Q: ppl 심하 네 / ppl 강하 네
A: 눈살 이 찌푸려 지 죠 . / 눈살 이 찌푸려 지 죠 .


In [74]:
que_corpus = que_corpus + arg_que_corpus + que_corpus
ans_corpus = ans_corpus + ans_corpus + arg_ans_corpus

In [75]:
len(que_corpus), len(ans_corpus)

(35469, 35469)

### Step 5. 데이터 벡터화
타겟 데이터인 `ans_corpus` 에 `<start>` 토큰과 `<end>` 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다.

In [76]:
a = ['i', 'love', 'you']
a = ['<start>'] + a + ['<end>']
a

['<start>', 'i', 'love', 'you', '<end>']

In [77]:
def add_tokens(sentence):
    added_tokens = []
    for list in sentence:
        temp_list = ['<start>'] + list + ['end>']

        added_tokens.append(temp_list)
    
    return added_tokens

ans_corpus = add_tokens(ans_corpus)

In [78]:
ans_corpus[:5]

[['<start>', '하루', '가', '또', '가', '네요', '.', 'end>'],
 ['<start>', '위로', '해', '드립니다', '.', 'end>'],
 ['<start>', '여행', '은', '언제나', '좋', '죠', '.', 'end>'],
 ['<start>', '여행', '은', '언제나', '좋', '죠', '.', 'end>'],
 ['<start>', '눈살', '이', '찌푸려', '지', '죠', '.', 'end>']]